In [1]:
import random
import pandas as pd
from sklearn.model_selection import StratifiedKFold
data_X=pd.read_csv('x.csv',encoding='utf-8')
data_y=pd.read_csv('y.csv',encoding='utf-8')
index =list(data_X.index)
random.Random(0).shuffle(index)
num=int(len(index)/5*4)
x_train,y_train=data_X.iloc[index[:num],:].values,data_y.iloc[index[:num],:].values
x_test,y_test=data_X.iloc[index[num:],:].values,data_y.iloc[index[num:],:].values
def get_batch(x_train,y_train):
    _index_in_epoch = 0
    for i in range(10000):
        start = _index_in_epoch
        _index_in_epoch += batch_size
        if _index_in_epoch > x_train.shape[0]:
            index = list(range(x_train.shape[0]))
            random.Random(0).shuffle(index)
            x_train = x_train[index]
            y_train = y_train[index]
            start = 0
            _index_in_epoch = batch_size
        end = _index_in_epoch
        batch_xs = x_train[start:end]
        batch_ys = y_train[start:end]
        yield batch_xs,batch_ys

In [ ]:
from __future__ import division, print_function, absolute_import
import tensorflow as tf
import numpy as np

# Parameters
learning_rate = 0.001
training_epochs = 2000
batch_size = 128
BETA = tf.constant(0.1)
RHO = tf.constant(0.05)
n_outputs=10
# Network Parameters
n_hidden_1 = 100 # Second layer num features
n_input =1200 # MNIST data input (img shape: 28*28)
# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])
trains=tf.placeholder(tf.bool)
drop_out_prob=tf.placeholder("float")
weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([n_input])),
}


def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                  biases['encoder_b1']))
    return layer_2
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    return layer_3
def KL_divergence(rho, rho_hat):
  one_minus_rho = tf.subtract(tf.constant(1.), rho)
  one_minus_rhohat = tf.subtract(tf.constant(1.), rho_hat)
  divergence = tf.add(log_func(rho, rho_hat), log_func(one_minus_rho, one_minus_rhohat))
  return divergence 
def log_func(x, y):
  return tf.multiply(x, tf.log(tf.div(x, y)))
x_norm = tf.layers.batch_normalization(X, center = True, scale = True, training = trains)
encoder_op = encoder(x_norm)
encoder_norm=tf.layers.batch_normalization(encoder_op,center = True, scale = True, training = trains)
decoder_op = decoder(encoder_norm)
num_batch = tf.placeholder("float", 1)
rho_hat = tf.div(tf.reduce_sum(encoder_norm, 0), num_batch)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X
# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cost_sparse = tf.multiply(BETA, tf.reduce_sum(KL_divergence(RHO, rho_hat)))
all_cost=tf.add(cost,cost_sparse)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(tf.add(cost, cost_sparse))

xs = tf.placeholder(tf.float32, [None, n_hidden_1])
Ws = tf.Variable(tf.zeros([n_hidden_1, n_outputs]))
bs = tf.Variable(tf.zeros([n_outputs]))

ys = tf.nn.softmax(tf.matmul(xs, Ws) +bs)
ys_ = tf.placeholder(tf.float32, [None, n_outputs])
cross_entropys = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=ys, labels=ys_))
lr=0.001
train_steps  = tf.train.AdamOptimizer(learning_rate = lr).minimize(cross_entropys)


correct = tf.equal(tf.argmax(ys, 1), tf.argmax(ys_, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
        sess.run(init)
        _index_in_epoch = 0
        for i in range(2000):
            # 清洗数据
            tmp_batch=get_batch(data_X.values,data_y.values)
            batch_xs,batch_ys=next(tmp_batch)
            # 定义每次送入网络的数据
            feed_dict1 = {X: batch_xs,num_batch: np.array([batch_size]),trains:True}
            _, c, c_sparse = sess.run([optimizer, cost, cost_sparse],feed_dict=feed_dict1)
            if i % 500 == 0:
                print(f'time_{i} is {c}')
                feed_dict2={X:x_test,num_batch: np.array([len(x_test)]),trains:True}
        all_cost_=sess.run(cost,feed_dict=feed_dict2)
        print(f'*****cost*** is {all_cost_}')
        print('***********finish_autoencoding*********')
        print('***********start_softmax*********')
        x_trains=sess.run(encoder_norm,feed_dict={X:x_train,trains:True})
        x_tests=sess.run(encoder_norm,feed_dict={X:x_test,trains:True})
        for i in range(5000):
            # 清洗数据
            tmp_batch=get_batch(x_trains,y_train)
            batch_xs,batch_ys=next(tmp_batch)
            feed_dict1 = {xs: batch_xs,ys_:batch_ys}
            _,costs = sess.run([train_steps,cross_entropys],feed_dict=feed_dict1)
            if i %1000 == 0:
                print(f'time_{i} is {costs}')
                feed_dict2={xs:x_tests,ys_:y_test}
                all_cost_=sess.run(accuracy,feed_dict=feed_dict2)
                print(f'*****cost*** is {all_cost_}')

In [84]:
a=get_batch(x_train,y_train)

In [ ]:
next(a)[0].shape